In [ ]:
### PACKAGES SETUP ###
import requests
from bs4 import BeautifulSoup
import time
import csv
import random
import os

In [3]:
### THE METACRITIC SCRAPER ###

base_url = "https://www.metacritic.com/browse/game/?releaseYearMin=1958&releaseYearMax=2024&page="

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'
}

rows = []

last_scraped_page = 0
last_game_name = None

if os.path.exists('MC_DF3.csv'):
    with open('MC_DF3.csv', 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        rows = list(reader)
        if len(rows) > 1:
            last_game_name = rows[-1][0]
            last_scraped_page = (len(rows) - 1) // 50 + 1

games_data = []

num_pages = 556

counter = len(rows) - 1

for page in range(last_scraped_page, num_pages + 1):
    print(f"Scraping page {page} of {num_pages}...")
    try:
        response = requests.get(base_url + str(page), headers=headers, timeout=10)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Website isn't responding: {e}")
        break
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    games = soup.find_all('div', class_='c-finderProductCard')
    
    for game in games:
        title_container = game.find('h3', class_='c-finderProductCard_titleHeading')
        name = title_container.find_all('span')[1].get_text(strip=True)

        score = game.find('div', class_='c-siteReviewScore').get_text(strip=True)
        
        release_date = game.find('div', class_='c-finderProductCard_meta').find('span').get_text(strip=True)
        
        games_data.append([name, score, release_date])
        counter += 1
        
        if counter % 50 == 0:
            with open('MC_DF3.csv', 'a', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                if counter == 50 + len(rows) - 1:
                    writer.writerow(['Name', 'Metacritic Score', 'Release Date'])
                writer.writerows(games_data)
            games_data = []
            print(f"Saved progress after {counter} games.")

    time.sleep(10)

if games_data:
    with open('MC_DF3.csv', 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerows(games_data)

print("Scraping completed. Time to relax!")


NameError: name 'os' is not defined